In [2]:
from autodp.mechanism_zoo import ExactGaussianMechanism, PureDP_Mechanism
from autodp.transformer_zoo import Composition
import matplotlib.pyplot as plt
from sklearn.datasets import load_svmlight_file

import time
import random
import numpy as np
from pathlib import Path
from phe import paillier

from util import  *

In [107]:
class OPE:
    def __init__(self):
        self.a = random.randint(1, 1<<16)
        self.b = random.randint(1, 1<<16)
        self.c = random.randint(1, 1<<16)

    def __call__(self, x):
        res = self.a * (x - self.c) ** 3 + self.b
        if x >= self.c:
            return res
        return 2 * self.b - res
        return 1.5 * x + 2

In [113]:
sigma1 = 5.0
sigma2 = 8.0

gm1 = ExactGaussianMechanism(sigma1,name='GM1')

compose = Composition()
composed_mech = compose([gm1], [3])

delta1 = 1e-4

In [126]:
# ds = read_libsvm('dna')
# xs = [i[0] for i in ds[0]]
xs = np.random.uniform(-10000, 10000, 1000)

In [124]:
# DP

st = time.time()

for x in xs:
    eps1 = composed_mech.get_approxDP(delta1)
    x += eps1

ed = time.time()

print(f'{(ed-st)*1000:.3f}')

1609.018


In [125]:
# Phaser

st = time.time()

pa = Phaser(xs, (max(xs)-min(xs)) * 0.5)
ope = OPE()
sorted(xs)
for x in xs:
    x = ope(pa(x))

ed = time.time()

print(f'{(ed-st)*1000:.3f}')

12.006


In [128]:
# HE

st = time.time()

pk, sk = paillier.generate_paillier_keypair()
for x in xs:
    _x = pk.encrypt(x)

ed = time.time()

print(f'{(ed-st)*1000:.3f}')

185922.755
